### **Cas Kaggle** - Àngela Obón Soto (1600359) i Muniba Liaqat Ali Ali (1635042)

El dataset que analitzarem conté informació sobre l'activitat dels llistats d'Airbnb a la ciutat de Nova York durant el 2019. Aquest conjunt de dades inclou detalls sobre els amfitrions, la ubicació geogràfica de les propietats, preus, disponibilitat, ressenyes i més.
**L'Objectiu Principal** d'aquesta anàlisi és explorar i comprendre els factors que influeixen en els preus i la disponibilitat de les llistes a Nova York, així com identificar patrons geogràfics i mètriques relacionades amb l'activitat dels amfitrions; i proporcionar recomanacions basades en l'anàlisi de dades.". En particular, busquem respondre les preguntes següents. 

1. Quines diferències hi ha en els preus entre les diferents àrees de la ciutat?
2. Quins factors tenen més influència en el preu dels llistats?
3. Quines àrees tenen la major disponibilitat i per què?
4. És possible identificar patrons a l'activitat dels amfitrions més ocupats?
5. Quins insights geoespacials es poden derivar de l'anàlisi de les dades?
6. Quins patrons s'observen a la distribució de preus entre diferents barris i tipus d'habitacions?
7. Quins són els barris més populars o més rendibles?
8. Quins factors tenen un impacte significatiu en la fixació de preus?

Aquesta anàlisi no només permet entendre les dinàmiques del mercat d'Airbnb a Nova York, sinó que també proporciona informació útil tant per a amfitrions que busquen optimitzar els llistats com per a usuaris interessats a trobar les millors opcions d'allotjament.

In [95]:
#imports

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import TargetEncoder


In [78]:
#exploracion inicial, veiem les dades del dataset

df = pd.read_csv('AB_NYC_2019.csv')
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [79]:
# Resumen extenso de less variables del dataset

def summarize_dataset(df):
    print("======================== ESTRUCTURA DEL DATASET ========================")
    print(df.info())
    
    print("\n======================== COLUMNAS EN EL DATASET ========================")
    print(df.columns.to_list())
    
    print("\n======================== TAMAÑO DEL DATASET ============================")
    print(f"Número de filas: {df.shape[0]}, Número de columnas: {df.shape[1]}")
    
    print("\n======================== PRIMERAS LÍNEAS DEL DATASET ===================")
    print(df.head())
    
    print("\n======================== VALORES NULOS ================================")
    nans = (df.isna().sum().sort_values() / len(df) * 100).round(2)
    print(nans)
    
    print("\n======================== TIPO DE DATOS ================================")
    print(df.dtypes)
    
    print("\n======================== DESCRIPCIÓN GENERAL ===========================")
    print(df.describe(include='all'))
    
    print("\n======================== VERIFICACIÓN DE DUPLICADOS ===================")
    print(f"Duplicados encontrados: {df.duplicated().sum()}")
    df = df.drop_duplicates()
    print("Duplicados eliminados.")
    

    
    return variables_summary_df

# Para usar la función, simplemente llamarla con el DataFrame:
# resumen_df = summarize_dataset(df)


In [80]:
resumen_df = summarize_dataset(df)

======================== ESTRUCTURA DEL DATASET ========================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews   

In [81]:

print("\n======================== RESUMEN DE VARIABLES =========================")
variables_summary = {
    "Column": df.columns,
    "Data Type": df.dtypes,
    "Null Values (%)": (df.isnull().sum() / len(df)) * 100,
    "Unique Values": df.nunique()
}

variables_summary_df = pd.DataFrame(variables_summary).reset_index(drop=True)

variables_summary_df


======================== RESUMEN DE VARIABLES =========================


,Column,Data Type,Null Values (%),Unique Values
0,id,int64,0.000000,48895
1,name,object,0.032723,47905
2,host_id,int64,0.000000,37457
3,host_name,object,0.042949,11452
4,neighbourhood_group,object,0.000000,5
5,neighbourhood,object,0.000000,221
6,latitude,float64,0.000000,19048
7,longitude,float64,0.000000,14718
8,room_type,object,0.000000,3
9,price,int64,0.000000,674


Hablar de mi objetivo, y justificar que voy a hacer con los nans

Tractament de NANS

El dataset té un nombre significatiu de registres (48,895 files originals), però alguns camps amb valors nuls, com last_review i reviews_per_month, no afecten directament altres columnes importants com price, room_type o neighbourhood.
Eliminar files amb NaNs podria reduir dràsticament la mida del dataset, cosa que podria disminuir la representativitat de les dades.

Variables com last_review o reviews_per_month són rellevants només per a llistats que han tingut ressenyes. Els valors nuls en aquestes columnes no impliquen un error, sinó que representen llistats sense activitat de ressenyes.
Transformar aquests nuls en valors com 0 o NaT reflecteix amb precisió que no hi ha dades disponibles per a aquestes mètriques en aquests casos.

Eliminar files amb NaNs podria esbiaixar l'anàlisi, ja que els llistats sense ressenyes podrien pertànyer a tipus de propietat, preus o ubicacions particulars.
Per exemple, els nous llistats (sense ressenyes) podrien representar tendències emergents en certes zones o categories d'Airbnb. Eliminar-los exclouria aquests casos de l'anàlisi.

Algunes columnes, com ara name o host_name, són descriptives i no afecten directament l'anàlisi numèrica. Omplir-les amb valors genèrics (No Name, Unknown) assegura que es puguin utilitzar per a anàlisis secundàries sense afectar el model principal.
Emplenar els valors de reviews_per_month amb 0 assegura que totes les observacions siguin utilitzables sense necessitat de dividir el dataset o aplicar filtres addicionals.

El dataset té com a objectiu explorar patrons de preus i activitat dels llistats d'Airbnb. Si bé algunes columnes amb valors nuls no són essencials per a totes les anàlisis, mantenir-les manejades adequadament millora la flexibilitat per respondre preguntes específiques.

In [83]:
#Identificar columnes amb valors nuls 

null_columns = df.columns[df.isnull().any()]
null_columns

Index(['name', 'host_name', 'last_review', 'reviews_per_month'], dtype='object')

In [84]:
# Maneig inicial de valors nuls:
# - 'name' i 'host_name': omplir amb un valor genéric
df['name'] = df['name'].fillna('No Name')
df['host_name'] = df['host_name'].fillna('Unknown')
# - 'last_review': Convertir a data, nuls es mantenen com a NaT
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')
# - 'reviews_per_month': Omplir nuls amb 0 (llistats sense ressenyes actives)
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

In [98]:
# Mostrar resultats
print("\nColumnas con valores nulos:")
print(null_columns)



Columnas con valores nulos:
Index(['name', 'host_name', 'last_review', 'reviews_per_month'], dtype='object')
